In [31]:
import tensorflow as tf
import numpy as np

In [32]:
x_inp = tf.placeholder(tf.float32, [5, 5])
w_inp = tf.placeholder(tf.float32, [3, 3])

x = tf.reshape(x_inp, [1, 5, 5, 1])
w = tf.reshape(w_inp, [3, 3, 1, 1])

x_valid = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="VALID")
x_same = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding="SAME")
x_valid_half = tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding="VALID")
x_same_half = tf.nn.conv2d(x, w, strides=[1, 2, 2, 1], padding="SAME")

In [33]:
x = np.array([[0, 1, 2, 1, 0],
             [4, 1, 0, 1, 0],
             [2, 0, 1, 1, 1],
             [1, 2, 3, 1, 0],
             [0, 4, 3, 2, 0]])
w = np.array([[0, 1, 0],
             [1, 0, 1],
             [2, 1, 0]])
sess = tf.Session()
y_valid, y_same, y_valid_half, y_same_half = sess.run(
    [x_valid, x_same, x_valid_half, x_same_half],
    feed_dict={x_inp: x, w_inp: w}
)

In [34]:
print("VALID:\n", y_valid[0, :, :, 0])
print("SAME:\n", y_same[0, :, :, 0])
print("VALID, stride2:\n", y_valid_half[0, :, :, 0])
print("SAME, stride2:\n", y_same_half[0, :, :, 0])

VALID:
 [[ 9.  5.  4.]
 [ 8.  8. 10.]
 [ 8. 15. 12.]]
SAME:
 [[ 5. 11.  4.  3.  3.]
 [ 3.  9.  5.  4.  4.]
 [ 5.  8.  8. 10.  3.]
 [ 4.  8. 15. 12.  6.]
 [ 5.  5.  9.  4.  2.]]
VALID, stride2:
 [[ 9.  4.]
 [ 8. 12.]]
SAME, stride2:
 [[5. 4. 3.]
 [5. 8. 3.]
 [5. 9. 2.]]


In [35]:
x_inp = tf.placeholder(tf.float32, [4, 4])
x = tf.reshape(x_inp, [1, 4, 4, 1])
x_valid = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding="VALID")
x_valid_half = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")

In [36]:
x = np.array([[0, 1, 2, 1],
            [4, 1, 0, 1],
            [2, 0, 1, 1],
            [1, 2, 3, 1]])
y_valid, y_valid_half = sess.run(
    [x_valid, x_valid_half],
    feed_dict={x_inp: x}
)
print("VALID:\n", y_valid[0, :, :, 0])
print("VALID, stride2:\n", y_valid_half[0, :, :, 0])

VALID:
 [[4. 2. 2.]
 [4. 1. 1.]
 [2. 3. 3.]]
VALID, stride2:
 [[4. 2.]
 [2. 3.]]


In [38]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [39]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))
conv_1 = tf.nn.conv2d(x_image, W_conv_1, strides=[1, 1, 1, 1], padding="SAME") + b_conv_1
h_conv_1 = tf.nn.relu(conv_1)
h_pool_1 = tf.nn.max_pool(h_conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

W_conv_2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv_2 = tf.nn.conv2d(h_pool_1, W_conv_2, strides=[1, 1, 1, 1], padding="SAME") + b_conv_2
h_conv_2 = tf.nn.relu(conv_2)
h_pool_2 = tf.nn.max_pool(h_conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64])
W_fc_1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)
keep_probability = tf.placeholder(tf.float32)
h_fc_1_drop = tf.nn.dropout(h_fc_1, keep_probability)

W_fc_2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1, shape=[10]))
logit_conv = tf.matmul(h_fc_1_drop, W_fc_2) + b_fc_2
y_conv = tf.nn.softmax(logit_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit_conv, labels=y))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [40]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_probability: 1.}))

0.9915


In [46]:
# выведите изображения которые были распознаны неверно(и их метки), все ли из них вы смогли распознать?

In [22]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [23]:
batch_size, img_rows, img_cols = 64, 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [26]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, border_mode="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode="same"))
model.add(Convolution2D(64, 5, 5, border_mode="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode="same"))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(28, 28, 1..., padding="same")`
  
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  after removing the cwd from sys.path.
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(28, 28, 1..., padding="same")`
  """
C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  import sys


In [27]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=10, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print("Test score: %f" % score[0])
print("Test accuracy: %f" % score[1])

Instructions for updating:
Use tf.cast instead.


C:\Users\Avsenin.Pavel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.1293 - acc: 0.9592 - val_loss: 0.0382 - val_acc: 0.9871
Epoch 2/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0433 - acc: 0.9861 - val_loss: 0.0335 - val_acc: 0.9882
Epoch 3/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0317 - acc: 0.9902 - val_loss: 0.0277 - val_acc: 0.9904
Epoch 4/10
60000/60000 [==============================] - 108s 2ms/step - loss: 0.0244 - acc: 0.9927 - val_loss: 0.0258 - val_acc: 0.9912
Epoch 5/10
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0194 - acc: 0.9934 - val_loss: 0.0295 - val_acc: 0.9915
Epoch 6/10
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0160 - acc: 0.9950 - val_loss: 0.0256 - val_acc: 0.9931
Epoch 7/10
60000/60000 [==============================] - 107s 2ms/step - loss: 0.0148 - acc: 0.9951 - val_loss: 0.0432 - 

In [47]:
batch_size = 64
latent_space = 128
learning_rate = 0.1
ae_weights = {"encoder_w": tf.Variable(tf.truncated_normal([784, latent_space], stddev=0.1)),
              "encoder_b": tf.Variable(tf.truncated_normal([latent_space], stddev=0.1)),
              "decoder_w": tf.Variable(tf.truncated_normal([latent_space, 784], stddev=0.1)),
              "decoder_b": tf.Variable(tf.truncated_normal([784], stddev=0.1))}
ae_input = tf.placeholder(tf.float32, [batch_size, 784])
hidden = tf.nn.sigmoid(tf.matmul(ae_input, ae_weights["encoder_w"]) + ae_weights["encoder_b"])
visible_logits = tf.matmul(hidden, ae_weights["decoder_w"]) + ae_weights["decoder_b"]
visible = tf.nn.sigmoid(visible_logits)
ae_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits, labels=ae_input))
optimizer = tf.train.AdagradOptimizer(learning_rate)
ae_op = optimizer.minimize(ae_cost)

In [84]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10000):
    x_batch, _ = mnist.train.next_batch(batch_size)
    sess.run(ae_op, feed_dict={ae_input: x_batch})

In [92]:
# выведите изображение 10 первых фильтров заданного автокодировщика

In [91]:
# выведите изображение первого фильтра в процессе обучения:
# обучайте модель на 2000 батчах, выводите изображение первого фильтра после каждого 100-го батча

In [50]:
# используя примеры из книги постройте разреженный, шумоподавляющий и сверточный автокодировщики
# выведите изображения фильтров этих автокодировщиков

In [ ]:
# !!! При выполнении заданий выше сюда не подсматривать !!!
import matplotlib.pyplot as plt

def showImage(images, index):
    first_image = images[index]
    first_image = np.array(first_image, dtype='float')
    pixels = first_image.reshape((28, 28))
    plt.imshow(pixels, cmap='gray')
    plt.show()

x_auto = sess.run(visible_logits, feed_dict={ae_input: mnist.test.images[0:batch_size]})
for i in range(10):
    showImage(x_auto, i)


import matplotlib.pyplot as plt

def showImage(images, index):
    first_image = images[index]
    first_image = np.array(first_image, dtype='float')
    pixels = first_image.reshape((28, 28))
    plt.imshow(pixels, cmap='gray')
    plt.show()

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    x_batch, _ = mnist.train.next_batch(batch_size)
    sess.run(ae_op, feed_dict={ae_input: x_batch})
    x_auto = sess.run(visible_logits, feed_dict={ae_input: x_batch})
    if i % 100 == 0:
        showImage(x_auto, 0)